# Parameters

In [1]:
dest_dir = "/tmp/ghe_20210701"

# Read data and reference tables

In [2]:
from owid import catalog
import json
from pathlib import Path

In [3]:
from etl.paths import BASE_DIR as base_path

In [4]:
reference_dataset = catalog.Dataset((base_path / "data/reference").as_posix())

In [5]:
sex = reference_dataset["sex"]

In [6]:
sex

,name
code,
female,Female
male,Male
both,Both sexes


In [7]:
countries = reference_dataset["countries_regions"]

In [8]:
countries.head()

,name,iso_alpha2,iso_alpha3,imf_code,cow_letter,cow_code,unctad_code,marc_code,ncd_code,kansas_code,penn_code,wikidata_uri,wikidata_label,legacy_entity_id,legacy_country_id,members
code,,,,,,,,,,,,,,,,
AFG,Afghanistan,AF,AFG,512.0,AFG,700.0,AFG,AF,AFGN,AFG,AFG,http://www.wikidata.org/entity/Q889,Afghanistan,15.0,562.0,NaN
ALA,Aland Islands,AX,ALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.wikidata.org/entity/Q5689,Åland,296.0,791.0,NaN
ALB,Albania,AL,ALB,914.0,ALB,339.0,ALB,AA,ALBN,ALB,ALB,http://www.wikidata.org/entity/Q222,Albania,16.0,565.0,NaN
DZA,Algeria,DZ,DZA,612.0,ALG,615.0,ALG,AE,ALGR,DZA,DZA,http://www.wikidata.org/entity/Q262,Algeria,17.0,619.0,NaN
ASM,American Samoa,AS,ASM,859.0,NaN,NaN,NaN,AS,NaN,ASM,ASM,http://www.wikidata.org/entity/Q16641,American Samoa,246.0,571.0,NaN


In [9]:
ghe_meadow = catalog.Dataset((base_path / "data/meadow/who/2021-07-01/ghe").as_posix())

In [10]:
ghe_estimates = ghe_meadow["estimates"]

In [11]:
ghe_estimates.shape

(7191900, 9)

In [12]:
ghe_estimates.head()

population  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders BTSX     YEARS45-49         6285.0   
                  Syphilis              MLE      YEARS1-4           3729.0   
                  Cataracts             FMLE     YEARS15-19         3736.0   
                  Peptic ulcer disease  BTSX     ALLAges          112002.0   
                  Drug use disorders    MLE      YEARS85PLUS         366.0   

                                                                deaths  \
country_code year ghe_cause_title       sex_code agegroup_code           
GRD          2019 Alcohol use disorders BTSX     YEARS45-49        0.5   
                  Syphilis              MLE      YEARS1-4          0.0   
                  Cataracts             FMLE     YEARS15-19        0.0   
                  Peptic ulcer disease  BTSX     ALLAges           4.7   
                  Drug use disorders    MLE      YEARS85PLUS       0.0   

                                                                 deaths_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders BTSX     YEARS45-49     8.200000e-05   
                  Syphilis              MLE      YEARS1-4       1.000000e-08   
                  Cataracts             FMLE     YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  BTSX     ALLAges        4.190000e-05   
                  Drug use disorders    MLE      YEARS85PLUS    8.524000e-05   

                                                                deaths_100k  \
country_code year ghe_cause_title       sex_code agegroup_code                
GRD          2019 Alcohol use disorders BTSX     YEARS45-49             8.2   
                  Syphilis              MLE      YEARS1-4               0.0   
                  Cataracts             FMLE     YEARS15-19             0.0   
                  Peptic ulcer disease  BTSX     ALLAges                4.2   
                  Drug use disorders    MLE      YEARS85PLUS            8.5   

                                                                 daly  \
country_code year ghe_cause_title       sex_code agegroup_code          
GRD          2019 Alcohol use disorders BTSX     YEARS45-49      42.1   
                  Syphilis              MLE      YEARS1-4         0.0   
                  Cataracts             FMLE     YEARS15-19       0.0   
                  Peptic ulcer disease  BTSX     ALLAges        126.1   
                  Drug use disorders    MLE      YEARS85PLUS      0.4   

                                                                   daly_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders BTSX     YEARS45-49     6.695200e-03   
                  Syphilis              MLE      YEARS1-4       5.900000e-07   
                  Cataracts             FMLE     YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  BTSX     ALLAges        1.126140e-03   
                  Drug use disorders    MLE      YEARS85PLUS    1.031580e-03   

                                                                daly_100k  \
country_code year ghe_cause_title       sex_code agegroup_code              
GRD          2019 Alcohol use disorders BTSX     YEARS45-49         669.5   
                  Syphilis              MLE      YEARS1-4             0.1   
                  Cataracts             FMLE     YEARS15-19           0.0   
                  Peptic ulcer disease  BTSX     ALLAges            112.6   
                  Drug use disorders    MLE      YEARS85PLUS        103.2   

                                                                causegroup  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders BTSX     YEARS45-49              2   
                  Syphilis          

In [13]:
country_codes = set(ghe_estimates.index.unique("country_code"))

In [14]:
len(country_codes)

183

In [15]:
owid_country_codes = set(countries.index)

In [16]:
len(owid_country_codes)

296

In [17]:
country_codes - owid_country_codes

set()

💡 The set of (country codes in GHE) minus (owid table country codes) is empty, so we don't need to do any further harmonization here.

In [18]:
ghe_estimates.index.unique("sex_code")

CategoricalIndex(['BTSX', 'MLE', 'FMLE'], categories=['BTSX', 'FMLE', 'MLE'], ordered=False, dtype='category', name='sex_code')

In [19]:
with open(
    base_path
    / "etl"
    / "steps"
    / "data"
    / "garden"
    / "who"
    / "2021-07-01"
    / "ghe.mapping.sex.json"
) as f:
    ghe_to_owid_sex_codes = json.load(f)

In [20]:
ghe_to_owid_sex_codes

{'BTSX': 'both', 'MLE': 'male', 'FMLE': 'female'}

In [21]:
index_names = ghe_estimates.index.names

In [22]:
ghe_estimates.reset_index(inplace=True)

In [23]:
ghe_estimates["sex_code"] = ghe_estimates["sex_code"].cat.rename_categories(
    ghe_to_owid_sex_codes
)

In [24]:
ghe_estimates.set_index(index_names, inplace=True)

In [25]:
ghe_estimates.head()

population  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders both     YEARS45-49         6285.0   
                  Syphilis              male     YEARS1-4           3729.0   
                  Cataracts             female   YEARS15-19         3736.0   
                  Peptic ulcer disease  both     ALLAges          112002.0   
                  Drug use disorders    male     YEARS85PLUS         366.0   

                                                                deaths  \
country_code year ghe_cause_title       sex_code agegroup_code           
GRD          2019 Alcohol use disorders both     YEARS45-49        0.5   
                  Syphilis              male     YEARS1-4          0.0   
                  Cataracts             female   YEARS15-19        0.0   
                  Peptic ulcer disease  both     ALLAges           4.7   
                  Drug use disorders    male     YEARS85PLUS       0.0   

                                                                 deaths_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders both     YEARS45-49     8.200000e-05   
                  Syphilis              male     YEARS1-4       1.000000e-08   
                  Cataracts             female   YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  both     ALLAges        4.190000e-05   
                  Drug use disorders    male     YEARS85PLUS    8.524000e-05   

                                                                deaths_100k  \
country_code year ghe_cause_title       sex_code agegroup_code                
GRD          2019 Alcohol use disorders both     YEARS45-49             8.2   
                  Syphilis              male     YEARS1-4               0.0   
                  Cataracts             female   YEARS15-19             0.0   
                  Peptic ulcer disease  both     ALLAges                4.2   
                  Drug use disorders    male     YEARS85PLUS            8.5   

                                                                 daly  \
country_code year ghe_cause_title       sex_code agegroup_code          
GRD          2019 Alcohol use disorders both     YEARS45-49      42.1   
                  Syphilis              male     YEARS1-4         0.0   
                  Cataracts             female   YEARS15-19       0.0   
                  Peptic ulcer disease  both     ALLAges        126.1   
                  Drug use disorders    male     YEARS85PLUS      0.4   

                                                                   daly_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders both     YEARS45-49     6.695200e-03   
                  Syphilis              male     YEARS1-4       5.900000e-07   
                  Cataracts             female   YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  both     ALLAges        1.126140e-03   
                  Drug use disorders    male     YEARS85PLUS    1.031580e-03   

                                                                daly_100k  \
country_code year ghe_cause_title       sex_code agegroup_code              
GRD          2019 Alcohol use disorders both     YEARS45-49         669.5   
                  Syphilis              male     YEARS1-4             0.1   
                  Cataracts             female   YEARS15-19           0.0   
                  Peptic ulcer disease  both     ALLAges            112.6   
                  Drug use disorders    male     YEARS85PLUS        103.2   

                                                                causegroup  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders both     YEARS45-49              2   
                  Syphilis          

In [26]:
ghe_garden = catalog.Dataset.create_empty(dest_dir)

In [27]:
ghe_garden.metadata = ghe_meadow.metadata
ghe_garden.save()

In [28]:
ghe_garden.add(ghe_estimates)

In [29]:
ghe_garden.save()